# Task 6.1 Set Estimator Options

## Objective 1: Estimator Options

The `Options` class for EstimatorV2 provides various configuration settings to `Estimator` execution .It impacts performance, accuracy, and resource usage.

### Attributes


| Attribute | Description | Notes |
|-----------|-------------|----------------|
| **`_VERSION`** | Internal version tracking |  |
| **`max_execution_time`** | Maximum time (seconds) for job execution |  |
| **`environment`** | Execution environment metadata | `EnvironmentOptions` object |
| **`simulator`** | Simulator-specific options | `{'noise_model': None, 'seed_simulator': None}` |
| **`default_precision`** | Target precision for expectation values |  Float default is `0.015625` (1/√4096)  |
| **`default_shots`** | Default number of circuit executions | Integer , no default value |
| **`resilience_level`** | Level of error mitigation | `0` (none) to `2` (medium) |
| **`seed_estimator`** | Random seed for reproducible results | Integer |
| **`dynamical_decoupling`** | Enable/configure dynamical decoupling | `DynamicalDecouplingOptions` object |
| **`resilience`** | Advanced resilience configuration | `Options.ResilienceOptionsV2` object |
| **`execution`** | Execution-specific settings | `ExecutionOptionsV2 ` object |
| **`twirling`** | Gate twirling configuration | `Options.TwirlingOptions` object |
| **`experimental`** | Experimental features | Dictionary of experimental settings |

In [ ]:
# Example: Configuring Estimator Options

from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, EstimatorOptions as Options
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# Initialize service (assuming credentials is already saved)
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Create a simple test circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
observable = SparsePauliOp("ZZ")

# Create Options object with custom settings
options = Options()

# Set basic execution parameters
options.max_execution_time = 300
options.default_shots = 2048 
options.default_precision = 0.01
options.seed_estimator = 12345

# Set resilience level (0=none, 1=basic, 2=moderate, 3=maximum)
options.resilience_level = 2  # moderate error mitigation

print(f"Max execution time: {options.max_execution_time} seconds")
print(f"Default shots: {options.default_shots}")
print(f"Default precision: {options.default_precision}")
print(f"Resilience level: {options.resilience_level}\n")

# Create Estimator with custom options
estimator = Estimator(mode=backend, options=options)

print("Updating Options\n")

# Create initial options
initial_options = Options()
initial_options.default_shots = 1024
initial_options.max_execution_time = 300

print(f"Initial shots: {initial_options.default_shots}")
print(f"Initial max time: {initial_options.max_execution_time}")

# Update options with new values
initial_options.update(default_shots=4096, max_execution_time=900)

print(f"Updated shots: {initial_options.default_shots}")
print(f"Updated max time: {initial_options.max_execution_time}\n")

### Methods


#### update

Updates multiple options at once. This is useful for modifying options after creating an Estimator instance or for applying configuration changes.

In [ ]:
options = Options()
options.update(
    default_shots=4096,
    max_execution_time=1200,
    resilience_level=2
)

## Objective 2: Twirling Options

Twirling (or randomized compiling) is a technique that averages over different but equivalent gate sequences to transform coherent errors into stochastic errors, which are easier to mitigate.

| Attribute | Description | Notes |
|-----------|-------------|----------------|
| **`enable_gates`** | enable 2-qubit clifford gate twirling or not | bool, default `False` |
| **`enable_measure`** | enable twirling to measumrent instruction or not | `True` for Estimator, `False` for Sampler |
| **`num_randomizations`** | number of random samples to use when twirling | Integer or 'auto' |
| **`shots_per_randomization`** | number of shots to run for each random sample. | Integer or 'auto' |
| **`strategy`** | strategy of twirling qubits | `active`, `active-circuit`, `active-accum`, `all`, default `active-accum`|


## Objective 3: Resilience Options

Resilience options control error mitigation techniques in `EstimatorV2`. These settings help improve the accuracy of quantum computations on noisy hardware.


### Attributes


| Attribute | Description | Noes |
|-----------|-------------|---------|
| **`measure_mitigation`** | Corrects measurement errors | default `True` |
| **`measure_noise_learning`** | Learns measurement noise model | `MeasureNoiseLearningOptions` object |
| **`zne_mitigation`** | Enables Zero-Noise Extrapolation | default `False` |
| **`zne`** | ZNE-specific configuration | `ZneOptions` object |
| **`pec_mitigation`** | Enables Probabilistic Error Cancellation | default `False` |
| **`pec`** | PEC-specific configuration | `PecOptions` |
| **`layer_noise_learning`** | Learns layer-wise noise |  `LayerNoiseLearningOptions` object|
| **`layer_noise_model`** | Custom layer noise model |  `NoiseLearnerResult` or `Sequence[LayerError]`  |


In [ ]:
from qiskit_ibm_runtime.options import MeasureNoiseLearningOptions,LayerNoiseLearningOptions,ZneOptions

# Example: Configuring Resilience Options


# Method 1: Using resilience_level
levels = [0, 1, 2]
descriptions = [
    "No error mitigation",
    "Minimal measurement error mitigation",
    "Medium mitigation"
]

for level, desc in zip(levels, descriptions):
    options = Options()
    options.resilience_level = level
    print(f"Level {level}: {desc}")
    
# Method 2: Control with Resilience Options

options = Options()

# Configure individual resilience techniques
# Basic readout correction
options.resilience.measure_mitigation = True
# Learn measurement noise
options.resilience.measure_noise_learning = MeasureNoiseLearningOptions()
# Enable Zero-Noise Extrapolation
options.resilience.zne_mitigation = True
# Disable PEC (computationally expensive)
options.resilience.pec_mitigation = False
# Learn layer-dependent noise
options.resilience.layer_noise_learning = LayerNoiseLearningOptions() 

# Configure ZNE specifically
options.resilience.zne = ZneOptions(noise_factors=[1.0, 2.0, 3.0], extrapolator="exponential")

print("configured resilience settings:")
print(f"  Measurement mitigation: {options.resilience.measure_mitigation}")
print(f"  ZNE mitigation: {options.resilience.zne_mitigation}")
print(f"  PEC mitigation: {options.resilience.pec_mitigation}")
print(f"  ZNE noise factors: {options.resilience.zne.noise_factors}")
print(f"  ZNE extrapolator: {options.resilience.zne.extrapolator}\n")


# Phase 1: minimal resilience
quick_test = Options()
quick_test.resilience_level = 0
quick_test.default_shots = 256
print(f"  Resilience: {quick_test.resilience_level} (none)")
print(f"  Shots: {quick_test.default_shots}")

# Phase 2: moderate resilience
moderate = Options()
moderate.resilience_level = 1
moderate.default_shots = 1024
print(f"  Resilience: {moderate.resilience_level} (basic)")
print(f"  Shots: {moderate.default_shots}")

# Phase 3: High resilience
high = Options()
high.resilience_level = 2
high.default_shots = 4096
print(f"  Resilience: {high.resilience_level} (advanced)")
print(f"  Shots: {high.default_shots}")

## Objective 4: ZNE Options

Zero-Noise Extrapolation (ZNE) is an error mitigation technique that:
1. Runs the circuit at amplified noise levels
2. Measures results at different noise factors
3. Extrapolates back to the zero-noise limit

### Attributes


| Attribute | Description | Options |
|-----------|-------------|---------|
| **`amplifier`** | Noise amplification method | `gate_folding`, `gate_folding_front`, `gate_folding_back`, `pea` |
| **`noise_factors`** | Noise amplification levels | List of floats (e.g., `[1.0, 2.0, 3.0]`) |
| **`extrapolator`** | Extrapolation method | `linear`, `exponential`, `double_exponential`, `polynomial_degree_1`, `polynomial_degree_2`, `polynomial_degree_3`, `polynomial_degree_4`, `polynomial_degree_5`, `polynomial_degree_6`, `polynomial_degree_7`, `fallback` |
| **`extrapolated_noise_factors`** | Used for fitting | Derived from noise_factors |

In [ ]:
# Example: Configuring ZNE Options

print("=== Zero-Noise Extrapolation (ZNE) Configuration ===\n")

# First, enable ZNE at the resilience level
options = Options()
options.resilience.zne_mitigation = True

# Configure ZNE parameters
print("ZNE Configuration Examples:\n")

# Basic ZNE with gate folding
options.resilience.zne.amplifier = "gate_folding"
options.resilience.zne.noise_factors = [1.0, 2.0, 3.0]
options.resilience.zne.extrapolator = "linear"

print(f"Case 1:")
print(f"  Amplifier: {options.resilience.zne.amplifier}")
print(f"  Noise factors: {options.resilience.zne.noise_factors}")
print(f"  Extrapolator: {options.resilience.zne.extrapolator}")

# Advanced ZNE for better accuracy
options.resilience.zne.amplifier = "gate_folding"
options.resilience.zne.noise_factors = [1.0, 1.5, 2.0, 3.0, 4.0]
options.resilience.zne.extrapolator = "exponential"

print(f"Case 2:")
print(f"  Amplifier: {options.resilience.zne.amplifier}")
print(f"  Noise factors: {options.resilience.zne.noise_factors}")
print(f"  Extrapolator: {options.resilience.zne.extrapolator}")

# Create ZNE configuration
zne_config = Options()

# Enable and configure ZNE
zne_config.resilience.zne_mitigation = True
zne_config.resilience.zne.amplifier = "gate_folding"
zne_config.resilience.zne.noise_factors = [1.0, 2.0, 3.0, 4.0]
zne_config.resilience.zne.extrapolator = "polynomial_degree_1"  # Polynomial extrapolation


# Set shots and execution time for ZNE
zne_config.default_shots = 2048
zne_config.max_execution_time = 600  # 10 minutes

print("Complete ZNE Configuration:")
print(f"  ZNE enabled: {zne_config.resilience.zne_mitigation}")
print(f"  Amplifier: {zne_config.resilience.zne.amplifier}")
print(f"  Noise factors: {zne_config.resilience.zne.noise_factors}")
print(f"  Extrapolator: {zne_config.resilience.zne.extrapolator}")
print(f"  Shots per factor: {zne_config.default_shots}")
print(f"  Total estimated shots: {zne_config.default_shots * len(zne_config.resilience.zne.noise_factors)}")
print(f"  Max execution time: {zne_config.max_execution_time} seconds")

---

## Practice Questions

**1) What is the primary purpose of setting resilience_level in EstimatorOptions?**

A) To control circuit execution on the backend

B) To select the transpilation resilience optimization level

C) To enable and coordinate error mitigation techniques

D) To configure batching and session behavior

**Answer:**
<details> <br/>

C) `resilience_level` determines which error mitigation strategies (twirling, ZNE, ...) are enabled and how they are applied.

</details>

---

**2) What is the key difference between active and active-circuit twirling strategies?**

A) active twirls measurements, active-circuit does not twirl measurments

B) active applies twirling around instructions on each layer separtely, while active-circuit wraps instruction qubits in each layer

C) active accumulates twirling across layers, while active-circuit resets after each layer

D) active is only supported on simulators

**Answer:**
<details> <br/>

B) when using 'active' only the instruction qubits in each individual twirled layer will be twirled, while using 'active-circuit' twirls the union of all instruction qubits in the circuit in each twirled layer.

</details>

---

**3) Complete this code to configure ZNE with 5 noise factors and polynomial extrapolation:**

```
from qiskit_ibm_runtime import EstimatorOptions as Options

options = Options()

# Enable ZNE mitigation
options.resilience.______ = True

# Configure ZNE parameters
options.resilience.zne.amplifier = "gate_folding"
options.resilience.zne.______ = [1.0, 1.5, 2.0, 3.0, 4.0]
options.resilience.zne.extrapolator = "______"
```

Which option correctly fills all three blanks?

A) zne_enable, noise_levels, polynomial

B) zne, factors, polynomial_degree_2

C) zne_mitigation, noise_factors, polynomial_degree_2

D) enable_zne, noise_factors, poly_2

**Answer:**
<details> <br/>

C) Correct attributes names are `zne_mitigation`, `noise_factors`, `polynomial_degree_2`
</details>

----